In [1]:
import itertools
import sys
import math

from sklearn.model_selection import train_test_split, KFold


data_path = '/Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/data/'

In [105]:
%run /Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/lib.py

In [106]:
train = Data()
train.read(data_path, 'train')

In [25]:
def expand_grid(dictionary):
   return pd.DataFrame([row for row in itertools.product(*dictionary.values())], 
                       columns=dictionary.keys())

In [26]:
def rmse(x, y):
    n = len(x)
    return math.sqrt(1.0 * sum((x - y)**2) / n)

In [31]:
def cross_validation(model, data, n_folds=5, n_iters=5, seed=2707, verbose=1):
    scores = []
    user_ids = data.edu['uid'].unique()
    
    for iter in range(n_iters):
        kf = KFold(n_splits=n_folds, random_state=seed+iter, shuffle=True)
        print(kf)
        fold = 0
        for train_index, test_index in kf.split(user_ids):
            fold += 1
            if verbose >= 2:
                print('Prepare data: %s' % datetime.datetime.now())
                
            data_train = data.get(user_ids[train_index])
            data_test = data.get(user_ids[test_index])

            if verbose >= 2:
                print('Fit: %s' % datetime.datetime.now())
            model.fit(data_train)
            if verbose >= 2:
                print('Predict: %s' % datetime.datetime.now())

            pred = model.predict(data_test)
            if verbose >= 2:
                print('Auc: %s' % datetime.datetime.now())
            score = rmse(data_test.edu['age'].values, pred['res'].values)
            scores.append(score)

            #if verbose >= 1:
            print('%2d -%2d : %1.4f, mean=%1.4f' % (iter, fold, score, np.mean(scores)))
    return np.mean(scores)

In [107]:
grid = expand_grid({
    'boosting_type': ['gbdt'],
    'min_data_in_leaf': [25],
    'lambda_l2': [0.0],
    'num_leaves': [10],
    'learning_rate': [0.03],
    'feature_fraction': [1],
    'bagging_fraction': [1],
    'bagging_freq': [5],
    'num_boost_round': [200],
})
min_rmse = 1e10
best_params = None
for i, r in grid.iterrows():
    model = LgbModel({
        'boosting_type': r.boosting_type,
        'min_data_in_leaf': r.min_data_in_leaf,
        'lambda_l2': r.lambda_l2,
        'num_leaves': r.num_leaves,
        'learning_rate': r.learning_rate,
        'feature_fraction': r.feature_fraction,
        'bagging_fraction': r.bagging_fraction,
        'bagging_freq': r.bagging_freq,
        'num_boost_round': r.num_boost_round,
    })
    rr = cross_validation(model, train, n_folds=5, n_iters=1, verbose=0)
    if rr < min_rmse:
        min_rmse = rr
        best_params = r.copy()
    print(r)
    print(rr)
        

KFold(n_splits=5, random_state=2707, shuffle=True)


/usr/local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Starting train: 2021-07-26 19:14:30.429649
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000459 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1013
[LightGBM] [Info] Number of data points in the train set: 23575, number of used features: 14
[LightGBM] [Info] Start training from score 35.017179
 0 - 1 : 10.5463, mean=10.5463
Starting train: 2021-07-26 19:14:33.469933
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000477 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1015
[LightGBM] [Info] Number of data points in the train set: 23575, number of used features: 14
[LightGBM] [Info] Start training from score 34.965302
 0 - 2 : 10.5844, mean=10.5653
Starting train: 2021-07-26 19:14

In [101]:
print(min_rmse)
print(best_params)

10.453795276926407
boosting_type       gbdt
min_data_in_leaf      25
lambda_l2            0.0
num_leaves            10
learning_rate       0.03
feature_fraction       1
bagging_fraction       1
bagging_freq           5
num_boost_round      200
Name: 0, dtype: object


In [108]:
prod_model = LgbModel({
    'boosting_type': 'gbdt',
    'min_data_in_leaf': 25,
    'lambda_l2': 0.0,
    'num_leaves': 10,
    'learning_rate': 0.03,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'num_boost_round': 200,
})
prod_model.fit(train)
prod_model.model.save_model('/Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/models/lgb.txt')
prod_model.predict(train)

Starting train: 2021-07-26 19:14:46.225390
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000642 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1032
[LightGBM] [Info] Number of data points in the train set: 29469, number of used features: 14
[LightGBM] [Info] Start training from score 35.029149


,uid,res
0,356,41.606110
1,26356,39.007565
2,97,34.654457
3,319,35.628959
4,30288,31.868401
...,...,...
29464,33331,52.329417
29465,119788,40.782107
29466,97581,36.203584
29467,104038,67.949525


In [57]:
train.friends

{105728: array([104361,  10008,  20675,  89855,  91395,  34496,  92187,  40876,
         67960,  76300,  53948,  76447,  95427, 101750,  70637,  59851,
         87970,  14988,  75798,  34430,  49969,  38594,  43020,   3336,
         26114,  31662, 108658,  42217,   9834,  12264, 100773,  32829,
         25169,  72830, 104361,  92468, 109210,  27254,  91271, 112649,
         11095,   3997, 108658,  69607,     92,  76289,  31459,  77355,
         18095,  73565,  62132,  66208,  22975,  72803, 114868, 114091,
         94387,  50556,  23020,  82447,  11718,  28839,  89467,   5908,
         99339,  14988,  17209,  27344,  89882,  96011,  26580,  64399,
         20287,  76300,  73533,  99339,  90922,  12264,   6896,  95427,
          1141,  12438, 100351,  51989,  99732, 100773,  16947,  26114,
         32112, 111885,  27344,  79483, 117605,  95614,  16174,  53487,
        102103,  24052,  66208, 107374,  38115,  20754,  27388,  50815,
         92468,  27388,  95562,  49994,   1690,  95847, 

In [58]:
a = train.edu.copy()

In [87]:
[
 np.median([
     train.registered_year_by_uid[fr]
     for fr in train.friends.get(uid, [])
     if fr in train.registered_year_by_uid
 ])
 for uid in a['uid'].values
]

[2009.0,
 2012.0,
 2010.0,
 2011.0,
 2010.0,
 2013.0,
 2013.0,
 2010.0,
 2011.0,
 2010.0,
 2014.0,
 2012.0,
 2011.0,
 2010.0,
 2012.0,
 2019.0,
 2011.0,
 2015.5,
 2010.0,
 2013.0,
 2010.0,
 2011.0,
 2015.0,
 2014.0,
 2011.0,
 nan,
 nan,
 2010.0,
 nan,
 nan,
 2011.0,
 2012.5,
 2011.0,
 2014.0,
 2011.0,
 2011.0,
 2009.0,
 2015.0,
 2012.0,
 2018.0,
 2013.0,
 2014.5,
 2011.0,
 nan,
 2014.0,
 nan,
 2011.0,
 2018.0,
 2009.0,
 nan,
 2014.0,
 2010.0,
 2012.5,
 2008.5,
 2012.0,
 2010.0,
 2009.5,
 nan,
 2013.0,
 2018.0,
 2012.0,
 2014.0,
 2010.0,
 2010.0,
 2016.0,
 2011.0,
 2010.0,
 2009.5,
 2011.0,
 2013.0,
 2014.0,
 2016.5,
 2014.0,
 2010.0,
 2016.0,
 2013.0,
 2015.5,
 2011.0,
 2011.0,
 2011.0,
 2013.5,
 2010.0,
 2018.0,
 2011.0,
 2008.0,
 2014.0,
 nan,
 2010.5,
 2009.5,
 2015.0,
 2015.0,
 2011.0,
 2012.0,
 2013.0,
 2014.0,
 2015.5,
 2016.0,
 2010.0,
 2018.0,
 2011.0,
 nan,
 2017.0,
 2011.0,
 2012.0,
 nan,
 2014.5,
 nan,
 2009.0,
 2014.0,
 2013.0,
 2013.0,
 2012.0,
 2012.0,
 2010.0,
 nan,
 201

In [79]:
np.median

<function numpy.median(a, axis=None, out=None, overwrite_input=False, keepdims=False)>

In [83]:
np.median([])

nan

In [97]:
dir(train)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_prepare_friends',
 '_prepare_groups',
 'edu',
 'friends',
 'get',
 'groups',
 'read',
 'registered_year_by_uid']

In [104]:
'ag1e' in train.edu.columns

False

In [112]:
a = pd.read_csv(os.path.join(data_path, f'train.csv')).drop(['age'], 1)
a.to_csv(os.path.join(data_path, f'test.csv'), header=True, index=False)